In [1]:
import re
import spacy
from nltk.corpus import stopwords

class TextCleaners:
    """Designed for Inverted Indexing"""
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")
        self.stop_words = set(stopwords.words('english'))

    def normalize(self, raw_text):
        """Remove special characters and lowercase text"""
        return re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", raw_text.lower())

    def remove_stopwords(self, raw_text):
        """Remove stopwords"""
        words = [word for word in raw_text.split() if word not in self.stop_words]
        return " ".join(words)

    def lemmatize(self, raw_text):
        """Perform lemmatization, return as a list of strings"""
        doc = self.nlp(raw_text)
        return [token.lemma_ for token in doc]

    def clean(self, raw_text):
        """Clean text by normalizing, removing stopwords, and lemmatizing"""
        raw_text = self.normalize(raw_text)
        raw_text = self.remove_stopwords(raw_text)
        return self.lemmatize(raw_text)   


In [2]:
import sqlite3

class dataPipelines:
    """Class of function for Update / Remove data"""
    
    def __init__(self, database_file):
        """Input database file"""
        self.conn = sqlite3.connect(database_file)
        self.cursor = self.conn.cursor()
        self.createTable()
        
    def createTable(self):
        # Create table for keeping domain name of url and times of referenced to
        self.cursor.execute("CREATE TABLE IF NOT EXISTS Reference_Domain(Domain_Name, Ref_Count)")
        # Create a table for unique id for each url and list of all words in that url and list of url found on that page
        self.cursor.execute("CREATE TABLE IF NOT EXISTS web_Data(Web_ID, URL, All_Word, Ref_To)")
        # Create table for each word, number of documnet that conatain that word and dictionary of sorted key that are id of url and number of that word found on that link
        self.cursor.execute("CREATE TABLE IF NOT EXISTS Inverted_Index(Word, Document_Freq, Inverted_Dict)")

    def uncountRef(self, domain_name_list):
        """For uncount referenced domain"""
        for domain in domain_name_list:
            # query_check = f"UPDATE Reference_Domain SET Ref_Count = Ref_Count - 1 WHERE Domain_Name = '{domain}'"
            self.cursor.execute(f"UPDATE Reference_Domain SET Ref_Count = Ref_Count - 1 WHERE Domain_Name = '{domain}'")
            self.conn.commit()

            
#     def removeInvertedIndex(self, web_id, words):
#         """Remove id from indexing and reduce docsfreq"""

#         for word in words:
#             # Retrieve the current values of Document_Freq and Inverted_Dict
#             self.cursor.execute(f"SELECT Document_Freq, Inverted_Dict FROM Inverted_Index WHERE Word=?", (word,))
#             result = self.cursor.fetchone()
#             doc_freq, inverted_dict = result[0], result[1]

#             # Decrement the Document_Freq value
#             doc_freq -= 1

#             # Convert the Inverted_Dict string to a dictionary and remove the entry for the Web_ID
#             inverted_dict = eval(inverted_dict)
#             inverted_dict.pop(str(web_id), None)

#             # Update the values of Document_Freq and Inverted_Dict for the word
#             self.cursor.execute(f"UPDATE Inverted_Index SET Document_Freq=?, Inverted_Dict=? WHERE Word=?", (doc_freq, str(inverted_dict), word))

#         # Commit the changes to the database
#         self.conn.commit()
    
    def removeInvertedIndex(self, web_id, words):
        """Remove id from indexing and reduce docsfreq"""
        for word in words:
            self.cursor.execute("SELECT Inverted_Dict FROM Inverted_Index WHERE Word=?", (word,))
            inverted_dict = eval(self.cursor.fetchone()[0])
            inverted_dict.pop(web_id, None)
            self.cursor.execute(f"UPDATE Inverted_Index SET Document_Freq=Document_Freq-1, Inverted_Dict=? WHERE Word=?", (str(inverted_dict), word))
        self.conn.commit()

        
    def removeWebData(self, url):
        """Remove data from web_Data"""
        self.cursor.execute(f"DELETE FROM web_Data WHERE URL=?", (url,))
        self.conn.commit()

    # ==============================================================


    def getUniqueID(self):
        """function for unique unused ID for a website"""
        self.cursor.execute(f"SELECT MAX(Web_ID) FROM web_Data")
        max_id = self.cursor.fetchone()[0]
        next_id = 1 if max_id is None else max_id + 1
        self.cursor.execute(f"SELECT Web_ID FROM web_Data WHERE Web_ID = {next_id}")
        while self.cursor.fetchone() is not None:
            next_id += 1
        return next_id
    
    def fetch_data_by_url(self, url):
        """get data from row by url"""
        self.cursor.execute("SELECT Web_ID, URL, All_Word, Ref_To FROM web_Data WHERE URL=?", (url,))
        # Fetch the result
        result = self.cursor.fetchone()
        # Return the result
        return {
            'Web_ID' : result[0],
            'URL' : result[1],
            'All_Word' : result[2].split(' , '),
            'Ref_To' : result[3].split(' , ')
        }

    # ==============================================================

#     OKAY ================================================================================    
    # cursor.execute("CREATE TABLE IF NOT EXISTS Reference_Domain(Domain_Name, Ref_Count)")
    def updateReferenceDomain(self, domains):
        """Update reference domain receiving a list of domain"""
        for domain in domains:
            # Check if the domain already exists in the table
            self.cursor.execute(f"SELECT Ref_Count FROM Reference_Domain WHERE Domain_Name=?", (domain,))
            result = self.cursor.fetchone()
            
            if result:
                # If the domain already exists, increment the Ref_Count by 1
                ref_count = result[0] + 1
                self.cursor.execute(f"UPDATE Reference_Domain SET Ref_Count=? WHERE Domain_Name=?", (ref_count, domain))
            else:
                # If the domain doesn't exist, insert a new entry with Ref_Count set to 1
                self.cursor.execute(f"INSERT INTO Reference_Domain (Domain_Name, Ref_Count) VALUES (?, 1)", (domain,))
        
        # Commit the changes to the database
        self.conn.commit()
    
#     OKAY ================================================================================
    def updateWebData(self, web_id, url, all_words, ref_to):
        """Insert new url data into web_Data"""
        words = list(all_words.keys())
        all_words = " , ".join(words)
        ref_to = " , ".join(ref_to)
        
        self.cursor.execute(f"INSERT INTO web_Data (Web_ID, URL, All_Word, Ref_To) VALUES (?, ?, ?, ?)", (web_id, url, all_words, ref_to))
        self.conn.commit()
        
    
    # cursor.execute("CREATE TABLE IF NOT EXISTS Inverted_Index(Word, Document_Freq, Inverted_Dict)")
    def updateInvertedIndexing(self, web_id, word_list):
        word_count = {}
        for word in word_list:
            word_count[word] = word_count.get(word, 0) + 1
        for word, count in word_count.items():
            self.cursor.execute(f"SELECT Word, Inverted_Dict FROM Inverted_Index WHERE Word = '{word}'")
            result = self.cursor.fetchone()
            if result:
                inverted_dict = eval(result[1])
                inverted_dict[web_id] = count
                inverted_dict = str(inverted_dict)
                self.cursor.execute(f"UPDATE Inverted_Index SET Document_Freq = Document_Freq + 1, Inverted_Dict = '{inverted_dict}' WHERE Word = '{word}'")
            else:
                self.cursor.execute(f"INSERT INTO Inverted_Index (Word, Document_Freq, Inverted_Dict) VALUES ('{word}', 1, '{{{web_id}:{count}}}')")
        self.conn.commit()

    
    # def updateInvertedIndexing(self, web_id, word_list):
    #     word_count = {}
    #     for word in word_list:
    #         word_count[word] = word_count.get(word, 0) + 1
    #     for word, count in word_count.items():
    #         self.cursor.execute(f"SELECT Word FROM Inverted_Index WHERE Word = '{word}'")
    #         result = self.cursor.fetchone()
    #         if result:
    #             self.cursor.execute(f"UPDATE Inverted_Index SET Document_Freq = Document_Freq + 1, Inverted_Dict = Inverted_Dict || '{','.join([f'{{{web_id}:{count}}}' for web_id, count in word_count.items()])}' WHERE Word = '{word}'")
    #         else:
    #             self.cursor.execute(f"INSERT INTO Inverted_Index (Word, Document_Freq, Inverted_Dict) VALUES ('{word}', 1, '{','.join([f'{{{web_id}:{count}}}' for web_id, count in word_count.items()])}')")
    #     self.conn.commit()


        
    # method for terminate the connection
    def close(self):
        """Close the connection"""
        # commit the changes
        self.conn.commit()
        self.conn.close

In [3]:
import sqlite3
from urllib.parse import urlparse

class LinkCheckers:
    """Class for working on URLs"""
    
    def __init__(self, database_file):
        """Input Database file"""
        self.conn = sqlite3.connect(database_file)
        self.cursor = self.conn.cursor()
    
    def alreadyScrape(self, url_to_check):
        """Check whether url already scrape, Return in True or false"""

        # query_check = f"SELECT * FROM Web_Data WHERE URL='{url_to_check}'"
        self.cursor.execute(f"SELECT * FROM Web_Data WHERE URL='{url_to_check}'")
        result = self.cursor.fetchone()

        if result:
            return True
        else:
            return False

    def checkAccessibility(self, url):
        """Check Whether URL is still accessible"""
        try:
            response = requests.get(url)
            response.raise_for_status()
            return True
        except requests.exceptions.HTTPError as err:
            return False

    def compareDomains(self, url1, url2):
        """Compare two url domain"""
        domain1 = urlparse(url1).hostname
        domain2 = urlparse(url2).hostname
        return domain1 == domain2
    
    # method for terminate the connection
    def close(self):
        """Close the connection"""
        # commit the changes
        self.conn.commit()
        self.conn.close

In [4]:
import validators
from bs4 import BeautifulSoup
import requests
from urllib.parse import urlparse
import re

class pageScrapers:
    """Class for Scrape single page, Return dictionary URL, all backlinks and Raw Text"""
    def __init__(self):
        # Set header
        self.headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}        
        # Set allowed domain 
        # self.allowed_domain = [
        #     "artyt.me",
        #     "www.35mmc.com",
        #     "www.dpreview.com"
        # ]
    
    def get_raw_html(self, url):
        """get raw html soup obj"""
        # webReq = requests.get(url)
        # return requests.get(url, headers=self.headers)
        res_temp = requests.get(url, headers=self.headers)
        if res_temp.status_code == 200:
            return res_temp
        else:
            return None
    
    def scrape_raw_text(self, html_text):
        """Return raw text string from bs4 boject"""
        # return ' '.join([raw.text for raw in soup_obj.find_all(['h1', 'p'])])
        soup = BeautifulSoup(html_text, 'html.parser')
        return soup.get_text()
    
    def scrape_all_urls(self, html_text):
        soup = BeautifulSoup(html_text, 'html.parser')
        urls = []
        for link in soup.find_all('a'):
            url = link.get('href')
            if url and re.match("^(http://|https://)", url) and not re.search(".(jpg|jpeg|png|gif)$", url):
                urls.append(url)
        return list(set(urls))
    
    def scrape_page(self, url):
        """Return a dictionary of url, all unrepeated backlinks and raw text"""
        raw_soup_html = self.get_raw_html(url).text
        return {
            "url" : url,
            "backlinks" : self.scrape_all_urls(raw_soup_html),
            "rawText" : self.scrape_raw_text(raw_soup_html)
        }

In [5]:
from urllib.parse import urlparse

def get_domain(url):
    """Get domain name (example.com) from a url"""
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    if domain.startswith('www.'):
        domain = domain[4:]
    return domain

def word_frequency_dict(words_list):
    """Turn list of words into dictionary with word as key and frequency as value"""
    frequency_dict = {}
    for word in words_list:
        if word in frequency_dict:
            frequency_dict[word] += 1
        else:
            frequency_dict[word] = 1
    return frequency_dict


def updateLink(url):
    newid = dataPipeline.getUniqueID()
    raw_mat = pageScraper.scrape_page(url)
    
    all_backlinks = raw_mat["backlinks"]

    page_domain = get_domain(url)

    all_domain = [] # <<<<<<<<< All domain

    for link in all_backlinks:
        # get domain of url
        domain_temp = get_domain(link)
        if (domain_temp not in all_domain):
            all_domain.append(domain_temp)
            
    cleanText_list = TextCleaner.clean(raw_mat["rawText"])
    word_dict = word_frequency_dict(cleanText_list)
    sorted_word_dict = dict(sorted(word_dict.items()))
    
    dataPipeline.updateReferenceDomain(all_domain)
    dataPipeline.updateWebData(newid, url, sorted_word_dict, all_domain)
    dataPipeline.updateInvertedIndexing(newid, sorted_word_dict)
    
    return all_domain
    
def removeData(url):
    """Remove Data By URL"""
    print("Removing : ", url)
    temp_datarow = dataPipeline.fetch_data_by_url(url)
    
    dataPipeline.removeWebData(temp_datarow['URL'])
    dataPipeline.uncountRef(temp_datarow['Ref_To'])
    dataPipeline.removeInvertedIndex(temp_datarow['Web_ID'], temp_datarow['All_Word'])

def scrapeLevel(url_set, depth, depth_limit):
    if depth <= depth_limit:
        for starter in url_set:
            if (linkChecker.alreadyScrape(starter) == False):
                # scrape and update database and got all backlink
                all_backlink = updateLink(starter)
                print("Scraped: " + starter)
                scrapeLevel(all_backlink, depth+1, depth_limit)

            else:
                print("Already Scraped: " + starter)
                # check if still accessible
                if (linkChecker.checkAccessibility(starter) == False):
                    removeData(starter)
                else:
                    pass
    else:
        pass
    
# main program
if __name__ == "__main__":

    tinderURL = {
        "https://photographylife.com/reviews/fuji-x100f"
        # "https://www.dpreview.com/reviews/sony-a7rv-review?utm_source=self-desktop&utm_medium=marquee&utm_campaign=traffic_source"
        # "https://www.35mmc.com/02/02/2023/hedeco-lime-two-low-profile-shoe-mount-light-meter-review/",
        # "https://petapixel.com/2023/02/03/canon-usa-settles-with-employees-affected-by-2020-ransomware-attack/",
        # "https://www.35mmc.com/14/10/2021/pentax-iqzoom-928-review/"
    }

    db_path = "database_3.db"

    TextCleaner = TextCleaners()
    pageScraper = pageScrapers()
    linkChecker = LinkCheckers(db_path)
    dataPipeline = dataPipelines(db_path)
    
    depth_limit = 2
    scrapeLevel(tinderURL, 1, depth_limit)
        
    
    print("Done")

Scraped: https://photographylife.com/reviews/fuji-x100f


MissingSchema: Invalid URL 'photographylife.com': No scheme supplied. Perhaps you meant http://photographylife.com?